In [20]:
from __future__ import absolute_import
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, f1_score
import numpy as np
from imblearn.metrics import geometric_mean_score
import pandas as pd
from collections import Counter
from tqdm import tqdm
from sklearn import datasets
from sklearn.model_selection import train_test_split
from semisupervised.TSVM import S3VM
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler
from imblearn.under_sampling import NearMiss
%matplotlib inline

In [6]:
data_sd = pd.read_csv('final_dataset.csv')
unlabeled_sd = pd.read_csv('final_unlabeled.csv')
unlabeled_sd['Label']=-1

In [7]:
data_nosd = data_sd.drop(['Subdomain levels'], axis=1)
unlabeled_nosd = unlabeled_sd.drop(['Subdomain levels'], axis=1)

In [27]:
def specificity(y_true, y_pred): 
    tn = confusion_matrix(y_true, y_pred)[0, 0]
    fp = confusion_matrix(y_true, y_pred)[0, 1]
    neg = tn + fp
    return tn/neg

def normalize(x):
    return (x - np.min(x))/(np.max(x) - np.min(x))

def get_data(labeled, unlabeled):
    
    X = labeled.drop(['Domain', 'Label'], axis=1).to_numpy()
    y = labeled['Label'].to_numpy()
    

    X_nearmiss, y_nearmiss = NearMiss(sampling_strategy=0.25).fit_sample(X, y)
    
    X_nearmiss = normalize(X_nearmiss)
    
    X_train, X_test, y_train, y_test = train_test_split(X_nearmiss, y_nearmiss, test_size = 0.3333, random_state = 42)
    
    unl_lvl = unlabeled[:19904]
    
    X_unl = unl_lvl.drop(['Domain', 'Label'], axis=1).to_numpy()
    y_unl = unl_lvl['Label'].to_numpy()
    
    X_unl = normalize(X_unl)
    
    
    return X_train, y_train, X_unl, y_unl, X_test, y_test

label_X_train_sd, label_y_train_sd, unlabel_X_train_sd, unlabel_y_sd, X_test_sd, y_test_sd = get_data(data_sd, unlabeled_sd)
label_X_train_nosd, label_y_train_nosd, unlabel_X_train_nosd, unlabel_y_nosd, X_test_nosd, y_test_nosd = get_data(data_nosd, unlabeled_nosd)

In [ ]:
model = S3VM()
model.fit(np.vstack((label_X_train_sd, unlabel_X_train_sd)), np.append(label_y_train_sd, unlabel_y_sd))

# predict
predict = model.predict(X_test_sd)

d:\Anaconda3\lib\site-packages\numpy\matrixlib\defmatrix.py:69: PendingDeprecationWarning: the matrix subclass is not the recommended way to represent matrices or deal with linear algebra (see https://docs.scipy.org/doc/numpy/user/numpy-for-matlab-users.html). Please adjust your code to use regular ndarray.
  return matrix(data, dtype=dtype, copy=False)


In [26]:
len(label_X_train_sd)

19904

In [21]:
# metric
print("accuracy", accuracy_score(y_test_sd, predict))
print('Specificity', specificity(y_test_sd, predict))

accuracy 0.8860416038589086
Specificity 0.4575673633845379
